***`Business Context`***,

In today's highly competitive market, maximizing **customer loyalty** is a top priority for businesses. Understanding the key **drivers** that hold the most value for customers is crucial in developing effective strategies to enhance loyalty.
<br><br>
To gain **insights** into customer loyalty, a comprehensive dataset comprising multiple surveys conducted among clients has been compiled. The primary objective is to identify the key **features and factors** that significantly influence customer loyalty.<br>
By analyzing this data, businesses can make informed decisions and tailor their offerings to better meet customer needs, ultimately fostering stronger loyalty and long-term customer relationships.

*packages*

In [2]:
# general
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import os, yaml, requests, random, time
import numpy as np
from math import pi
from IPython.display import Image
import pickle

# shap
import shap 

# optimization
import pyomo.environ as pyo

# scikitlearn
from sklearn.model_selection import train_test_split
from sklearn import svm, metrics, decomposition
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import *

# To plot pretty figures
%matplotlib inline
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
plt.style.use('ggplot')

***`load the model`***

In [13]:
with open('model.pkl', 'rb') as file:
    model = pickle.load(file)

In [14]:
model

RandomForestClassifier(max_depth=10, min_samples_split=5, n_jobs=-1,
                       random_state=42)

In [15]:
print('estimator: ', model.estimator_)
print('classes: ', model.classes_)
print('features: ', model.feature_names_in_)
print('feature importances: ', model.feature_importances_)

estimator:  DecisionTreeClassifier()
classes:  [0 1]
features:  ['productRate' 'priceRate' 'promoRate' 'ambianceRate' 'wifiRate'
 'serviceRate' 'chooseRate']
feature importances:  [0.16051834 0.29423861 0.1143797  0.12877026 0.06616064 0.05527729
 0.18065516]


*load the data*

`train`

In [16]:
X_train = pd.read_csv('X_train.csv').rename(columns = {'Unnamed: 0':'index'}).set_index('index')
y_train = pd.read_csv('y_train.csv').rename(columns = {'Unnamed: 0':'index'}).set_index('index')

`test`

In [17]:
X_test = pd.read_csv('X_test.csv').rename(columns = {'Unnamed: 0':'index'}).set_index('index')
y_test = pd.read_csv('y_test.csv').rename(columns = {'Unnamed: 0':'index'}).set_index('index')

#### problem optimization

*`sets:`*
   - drivers: product, price, promo, ambiance, wifi, service, choose
   - surveyed: 113
   
*`parameters:`*
   - price increase
   - loyalty
   - betas

*`output:`*
   - loyalty increase

In [35]:
feature_labels = model.feature_names_in_
feature_betas = model.feature_importances_

In [51]:
# model
model_pyo = pyo.ConcreteModel()

model_pyo.sDrivers = pyo.Set(initialize = feature_labels)
model_pyo.sSurveyed = pyo.Set(initialize = [i for i in range(X_train.shape[0])])

In [52]:
df_perception = X_train.reset_index(drop = True, inplace = False)

In [53]:
df_perception.head()

,productRate,priceRate,promoRate,ambianceRate,wifiRate,serviceRate,chooseRate
0,8,6,6,6,4,6,8
1,10,2,6,6,4,6,6
2,6,4,8,6,6,6,8
3,8,8,10,8,4,6,8
4,10,4,10,10,4,8,6


In [54]:
model_pyo.pBetas = pyo.Param(model_pyo.sDrivers, initialize = dict(zip(feature_labels, feature_betas)))

In [55]:
model_pyo.pBetas.display()

pBetas : Size=7, Index=sDrivers, Domain=Any, Default=None, Mutable=False
    Key          : Value
    ambianceRate : 0.12877025843524212
      chooseRate :  0.1806551645660694
       priceRate :  0.2942386120604277
     productRate : 0.16051833695000436
       promoRate : 0.11437969881373418
     serviceRate : 0.05527729008139536
        wifiRate : 0.06616063909312678


In [56]:
def load_perception(model, survey, driver):
    return df_perception[driver][survey]

In [57]:
model_pyo.pPerception = pyo.Param(model_pyo.sSurveyed, 
                                  model_pyo.sDrivers, 
                                  initialize = load_perception)

In [63]:
for i in model_pyo.pPerception.items():
    print(i)
    break
#i, model_pyo.pPerception[('16', 'productRate')]

((0, 'productRate'), 8)
